## Initialize notebook

In [ ]:
import configparser
import glob
import os

import ipywidgets
import ipywidgets as widgets
import pandas as pd
import skimage.io

In [ ]:
%gui qt
import napari
viewer = napari.Viewer()

## Select image 👩🏻‍💻

In [ ]:
# Set analysis folder
dir_analysis = "./live_analysis/"

In [ ]:
# Read configuration file
config = configparser.ConfigParser()
config.read(os.path.join(dir_analysis, "luigi.cfg"))
do_3D = eval(config["General"]["do_3D"])
do_TimeSeries = eval(config["General"]["do_TimeSeries"])
spots_cols = ["frame", "y", "x"]
track_cols = ["particle", "frame", "y", "x"]

# Bead alignment
if eval(config["PreprocessingAlignment"]["enabled"]):
    alignment = skimage.io.imread(os.path.join(dir_analysis, "alignment.png"))
    viewer.add_image(alignment, name="Alignment")

In [ ]:
# Select from nd files
files = sorted(glob.glob(os.path.join(dir_analysis, "preprocessed", "*.tif")))
files = sorted([os.path.basename(f).replace(".tif", "") for f in files])
file = widgets.Dropdown(
    options=files, description="File:", layout=ipywidgets.Layout(width="100%")
)
file

## Load images 🌠

In [ ]:
viewer.layers.clear()

# Raw image
image = skimage.io.imread(
    os.path.join(dir_analysis, "preprocessed", f"{file.value}.tif")
)
for idx, channel in enumerate(image):
    viewer.add_image(channel, name=f"Channel {idx}", blending="additive")

# Segmentation primary
fname = os.path.join(dir_analysis, "segmentation_primary", f"{file.value}.tif")
segmap = skimage.io.imread(fname).astype(int)
viewer.add_labels(segmap, name="Segmentation Primary", blending="translucent")

# Segmentation secondary
if eval(config["SegmentationSecondary"]["enabled"]):
    fname = os.path.join(dir_analysis, "segmentation_secondary", f"{file.value}.tif")
    segmap = skimage.io.imread(fname).astype(int)
    viewer.add_labels(segmap, name="Segmentation Secondary", blending="translucent")

# Segmentation other
if eval(config["SegmentationOther"]["enabled"]):
    for channel in eval(config["SegmentationOther"]["channels"]):
        fname = os.path.join(
            dir_analysis, f"segmentation_{channel}", f"{file.value}.tif"
        )
        segmap = skimage.io.imread(fname).astype(int)
        viewer.add_labels(
            segmap, name=f"Segmentation C{channel}", blending="translucent"
        )

# Detection raw (or joined)
for channel in eval(config["SpotsDetection"]["channels"]):
    folder = (
        f"detection_final_c{channel}"
        if do_3D or do_TimeSeries
        else f"detection_raw_c{channel}"
    )
    fname = os.path.join(dir_analysis, folder, f"{file.value}.parq")
    df = pd.read_parquet(fname)
    if do_TimeSeries:
        viewer.add_tracks(df[track_cols], name=f"Track C{channel}")
    else:
        viewer.add_spots(df[spots_cols], name=f"Detection C{channel}")


# Colocalization
if eval(config["SpotsColocalization"]["enabled"]):
    for i, j in eval(config["SpotsColocalization"]["channels"]):
        fname = os.path.join(
            dir_analysis, f"colocalization_{i}-{j}", f"{file.value}.parq"
        )
        df = pd.read_parquet(fname)
        df_coloc = df[(df["coloc_particle"].isna()) & (df["channel"] == i)][track_cols]
        df_empty = df[(~df["coloc_particle"].isna()) & (df["channel"] == j)][track_cols]
        viewer.add_tracks(df_coloc, name=f"Track {i}-{j} Coloc", colormap="red")
        viewer.add_tracks(df_empty, name=f"Track {i}-{j} Empty", colormap="blue")